# Resize with adding smooth borders
This notebook has the sample code to resize imagese which are not square by adding pixels in rows or columns in a way to not introduce artifacts in the images.

The function `make_square_image` is responsible for 2 jobs:
- add pixels in the borders to make squared images
- resize the squared images into 128x128


### Imports

In [53]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import shutil

### Resize and add border function

In [23]:
def make_square_image(img):
    
    height, width, channels = img.shape
    x = height if height > width else width
    y = height if height > width else width
    
    #Padding
    y_padding = y-height
    
    x_padding = x-width

    
    kernal = (29,29)

    if y_padding > 0:
       
        
        row_top_3 = img[:3,:]
        row_bottom_3 = img[-3:,:]
        
        repeatition = math.ceil((width/2)/3)
    
        
        start = math.floor(y_padding/2)
        end = start+height
        
        
        #-------#-------#-------#-------#-------#-------#-------
   
        
            
        blurred_top = cv2.GaussianBlur(row_top_3,kernal,0)
        blurred_bottom = cv2.GaussianBlur(row_bottom_3,kernal,0)

        repeated_top = np.repeat(blurred_top,repeats=repeatition, axis=0)
        repeated_bottom = np.repeat(blurred_bottom,repeats=repeatition, axis=0)

        one_row_image_blurred_repeated = np.vstack((repeated_top,repeated_bottom))
        
        new_img_height = one_row_image_blurred_repeated.shape[0]
        new_img_width = one_row_image_blurred_repeated.shape[1]
        if new_img_height > new_img_width:
            one_row_image_blurred_repeated = one_row_image_blurred_repeated[:new_img_width,:]
            
            
        
        combined_img = one_row_image_blurred_repeated.copy()
        combined_img[start:end,:] = img


        #-------#-------#-------#-------#-------#-------#-------#-------
        
        
     
        
    
    elif x_padding > 0:
        
        col_left_3 = img[:,:3]
        col_right_3 = img[:,-3:]

       
        repeatition = math.ceil((height/2)/3)
    
        
        start = math.floor(x_padding/2)
        end = start+width
        
        
        #-------#-------#-------#-------#-------#-------#-------
   
       
            
        blurred_left = cv2.GaussianBlur(col_left_3,kernal,0)
        blurred_right = cv2.GaussianBlur(col_right_3,kernal,0)

        repeated_left = np.repeat(blurred_left,repeats=repeatition, axis=1)
        repeated_right = np.repeat(blurred_right,repeats=repeatition, axis=1)

        one_col_image_blurred_repeated = np.hstack((repeated_left,repeated_right))
        
        new_img_height = one_col_image_blurred_repeated.shape[0]
        new_img_width = one_col_image_blurred_repeated.shape[1]
        if new_img_width > new_img_height:
            one_col_image_blurred_repeated = one_col_image_blurred_repeated[:,:new_img_height]

        combined_img = one_col_image_blurred_repeated.copy()
        combined_img[:, start:end] = img

    else:
        combined_img = img
        
        
        #-------#-------#-------#-------#-------#-------#-------#-------
    
    resized_image = cv2.resize(combined_img, (128,128), interpolation = cv2.INTER_AREA)
    return resized_image
    
    

### Source/Destinatino directories and the CSV with image names

In [77]:
source_dir = "source_directory"
destination_dir = "destination_directory"
csv = "labels.csv"

### Load CSV filee

In [78]:
data = pd.read_csv(csv)
data.shape

### Loop over images
- Load image from souce directory
- Resize using the funciton
- Save the image to destination directory

### Note:
It is assumed the label column name is ***label*** and the file name column is ***file_name*** 

In [ ]:
problem_indexes = [] # for capturing the indexes of images which are not resize because of any error

for i,row in data.iterrows():
    
    read_file = os.path.join(source_dir, row.label, row.file_name)
    write_file = os.path.join(destination_dir, row.file_name)

    try:
        img = cv2.imread(read_file)
        sq_img = make_square_image(img)
        cv2.imwrite(write_file, sq_img)

    except:

        print("Error at index {}".format(i))
        problem_indexes.append(i)
        


### Check the problematic Indexes

In [ ]:
# print the array
problem_indexes

In [ ]:
# get the row from the dataframe with one index
row = data.loc[2460534] 
row